In [67]:
!pip install pymystem3

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'RECORD'
Consider using the `--user` option or check the permissions.



In [277]:
import re
import numpy as np
import pandas as pd
import requests
import jsonlines
import random

import gensim

from pymystem3 import Mystem

import pymorphy2
m = pymorphy2.MorphAnalyzer()

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import string
from string import punctuation

from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ekostrykina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [278]:
def get_data(filename):
    data = []
    with jsonlines.open(f'TERRa/{filename}.jsonl') as reader:
        for obj in reader:
            data.append(obj)
    return pd.DataFrame(data)

In [279]:
val_df = get_data('val')

In [63]:
url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

{'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


In [64]:
def tag_mystem(text):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

In [13]:
m_wv = "220/model.bin"
model = gensim.models.KeyedVectors.load_word2vec_format(m_wv, binary=True, unicode_errors='ignore')

In [352]:
from gensim.models import KeyedVectors

w2v_fpath = "news_upos_cbow_600_2_2018.vec"

w2v = KeyedVectors.load_word2vec_format(w2v_fpath, binary=False, unicode_errors='ignore')
w2v.init_sims(replace=True)

for word, score in w2v.most_similar(positive=["депутат_NOUN"], topn=5):
    print(word, score)

<ipython-input-352-074468d7233b>:6: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


парламентарий_NOUN 0.5742719173431396
нардеп_NOUN 0.5596699714660645
депутат_PROPN 0.4610869884490967
яблочник_NOUN 0.45993927121162415
единоросс_NOUN 0.45825690031051636


In [353]:
def synonym_attack(sentence):
    #выбираем слово
    text = word_tokenize(sentence)
    positions = [i for i in range(len(text))]

    while len(positions)>=1:
        position = random.choice(positions)
        positions.remove(position)
        word = text[position]

        while word in punctuation and len(positions)>=1:
            position = random.choice(positions)
            positions.remove(position)
            word = text[position]
        # сохраняем информацию о слове
        #m.parse(word)[0].tag.POS
        parsed_word = m.parse(word)[0]
        gender = parsed_word.tag.gender
        number = parsed_word.tag.number
        case = parsed_word.tag.case

        #находим синоним
        try:
            synonyme = w2v.most_similar(positive=tag_mystem(word), topn=1)[0][0]
            synonyme = synonyme.split('_')[0]

            # возвращаем грамматику
            inflected = synonyme
            try:
                inflected = m.parse(inflected)[0].inflect({gender})
            except:
                pass 

            try:
                if type(inflected) is str:
                    inflected = m.parse(inflected)[0].inflect({case})
                else:
                    inflected = m.parse(inflected.word)[0].inflect({case})
            except:
                pass

            try:
                if type(inflected) is str:
                    inflected = m.parse(inflected)[0].inflect({number})
                else:
                    inflected = m.parse(inflected.word)[0].inflect({number})
            except:
                pass

            synonyme_word = inflected.word
        except:
            continue
        if text[position] != synonyme_word:
            text[position] = synonyme_word
            text = ' '.join(text)

            # удаляем лишние пробелы
            pat = "\s+([{}]+)".format(re.escape(string.punctuation))
            output_text = re.sub("\s{2,}", " ", re.sub(pat, r"\1", text))
            return output_text, True, word, synonyme_word
    return sentence, False, None, None

In [354]:
synonym_attack('Красивая девочка, красивая кошка и собака.')

('Красивая девочка, эффектная кошка и собака.', True, 'красивая', 'эффектная')

In [306]:
synonym_attack('Красивая девочка, красивая кошка и собака.')

('Красивая девочка, красивая собака и собака.', True, 'кошка', 'собака')

In [363]:
synonym_attack('а о и')

('а о и', False, None, None)

In [288]:
synonym_attack('Красивая девочка, красивая кошка и собака.')

красивая


('Красивая девочка, прекрасная кошка и собака.',
 True,
 'красивая',
 'прекрасная')

In [364]:
attacked_data = []
flags = []
original_words = []
inflected_words = []

for i in tqdm(val_df['hypothesis']):
    sentence, flag, original_word, inflected_word = synonym_attack(i)
    attacked_data.append(sentence)
    flags.append(flag)
    original_words.append(original_word)
    inflected_words.append(inflected_word)

  0%|          | 0/307 [00:00<?, ?it/s]

In [365]:
sum(flags)

301

In [366]:
val_df['original_hypothesis'] = val_df['hypothesis']
val_df['hypothesis'] = attacked_data
val_df['flags'] = flags
val_df['original_words'] = original_words
val_df['inflected_words'] = inflected_words

In [367]:
val_df

,premise,hypothesis,label,idx,original_hypothesis,flags,original_words,inflected_words
0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получимся.,entailment,0,Взыскать компенсацию не получимся.,False,None,None
1,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,entailment,1,На улице Кеэвисе жителей эвакуировались из-за ...,True,эвакуировались,эвакуировали
2,"""В суде Центрального района Сочи завершилось р...",81-летний нетрудоспособный застрелил обидчика ...,entailment,2,81-летний иждивенец застрелил обидчика дочери.,True,иждивенец,нетрудоспособный
3,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то нарисовал на снегу контур человечества ...,entailment,3,Кто-то изобразил на снегу контур человечества ...,True,изобразил,нарисовал
4,"Также установлено, что работники учреждения не...",В десткие садике не все вовремя проходят медос...,entailment,4,В детском садике не все вовремя проходят медос...,True,детском,десткие
...,...,...,...,...,...,...,...,...
302,"Грабитель поджидал своих жертв и, угрожая им п...",У бандита был револьвер.,entailment,302,У бандита был пистолет.,True,пистолет,револьвер
303,Во время обыска в квартире подозреваемого поли...,С сентября прошлого года подозреваемый был на ...,entailment,303,С октября прошлого года подозреваемый был на с...,True,октября,сентября
304,"""""""После инцидента 15 августа 2011 г. ко мне п...",береговой не делал видеозапись.,not_entailment,304,береговой не сделал видеозапись.,True,сделал,делал
305,Сыграло ли это событие роль в решении обвиняем...,Дивенни мог работать на иностранную разведоват...,entailment,305,Дивенни мог работать на иностранную разведыват...,True,разведывательную,разведовательную


In [368]:
val_df.to_csv('attack11_val.csv', index=False)

In [369]:
new_val = pd.read_csv('new_val.csv')

In [370]:
attacked_data = []
flags = []
original_words = []
inflected_words = []

for i in tqdm(new_val['hypothesis']):
    sentence, flag, original_word, inflected_word = synonym_attack(i)
    attacked_data.append(sentence)
    flags.append(flag)
    original_words.append(original_word)
    inflected_words.append(inflected_word)

  0%|          | 0/923 [00:00<?, ?it/s]

In [371]:
sum(flags)

902

In [372]:
new_val['original_hypothesis'] = new_val['hypothesis']
new_val['hypothesis'] = attacked_data
new_val['flags'] = flags
new_val['original_words'] = original_words
new_val['inflected_words'] = inflected_words

In [373]:
new_val.to_csv('attack11_new_val.csv', index=False)

In [276]:
m.parse('гуляю')[0].inflect({'plur', 'femn'})

In [269]:
m.parse('гулял')[0].inflect({'femn'})

Parse(word='гуляла', tag=OpencorporaTag('VERB,impf,intr femn,sing,past,indc'), normal_form='гулять', score=1.0, methods_stack=((DictionaryAnalyzer(), 'гуляла', 15, 8),))

In [ ]:
if past:
    change gender 